# Imports

In [ ]:
from pathlib import Path

from imageio import imread
import geopandas as gpd
import matplotlib.pyplot as plt
import pyvista as pv
import rasterio as rio
from shapely import Point

from geograypher.cameras import MetashapeCameraSet
from geograypher.constants import DATA_FOLDER, INSTANCE_ID_KEY, LAT_LON_CRS
from geograypher.entrypoints.project_detections import project_detections
from geograypher.predictors.derived_segmentors import TabularRectangleSegmentor
from geograypher.utils.visualization import create_pv_plotter
from geograypher.utils.parsing import parse_transform_metashape

# Set constants

In [ ]:
# The data folder on Google Drive should be placed in the `data` subfolder of the repository
DETECTIONS_DATA_FOLDER = Path(DATA_FOLDER, "example_detection")
INPUT_FOLDER = Path(DETECTIONS_DATA_FOLDER, "inputs")
INTERMEDIATE_FOLDER = Path(DETECTIONS_DATA_FOLDER, "intermediate_results")
OUTPUT_FOLDER = Path(DETECTIONS_DATA_FOLDER, "outputs")

# Path to cameras
CAMERAS_FILENAME = Path(INPUT_FOLDER, "hidden_little_cameras.xml")
# Path to mesh
MESH_FILENAME = Path(INPUT_FOLDER, "hidden_little_mesh.ply")
# Path to images
IMAGE_FOLDER = Path(INPUT_FOLDER, "images")
# Path to detection predictions
DETECTIONS_FOLDER = Path(INPUT_FOLDER, "detections")
# Path to orthomosaic
ORTHO_FILENAME = Path(INPUT_FOLDER, "hidden_little_ortho.tif")

# Path to saved projections onto the face. Can either be read from or written to depending on the step
PROJECTIONS_TO_MESH_FILENAME = Path(INTERMEDIATE_FOLDER, "projections_to_mesh.npz")
# File to export the geospatial projections to
PROJECTIONS_TO_GEOSPATIAL_SAVEFILENAME = Path(
    OUTPUT_FOLDER, "detections_projected_to_geospatial.geojson"
)
# File to export the triangulated points to
TRIANGULATED_POINTS_SAVEFILE = Path(
    OUTPUT_FOLDER, "detections_triangulated_to_geospatial.geojson"
)

# Focal length of the camera in pixels
DEFAULT_FOCAL_LENGTH = 8688
# Whether to run the step for projecting images to meshes
PROJECT_TO_MESH = True
# Whether to run the conversion from mesh to geospatial
CONVERT_TO_GEOSPATIAL = True
# Whether to show the mesh
VIS_MESH = True
# Whether to show the geospatial predictions
VIS_GEODATA = True

# Keyword arguments for the segmentor
SEGMENTOR_KWARGS = {
    "image_path_key": "image_path",
    "label_key": "instance_ID",
    "split_bbox": False,
}
# For the ray-based approach, the tolerance between detections to consider them a match
RAY_BASED_SIMILARITY_THRESHOLD_METERS = 0.25
# The resolution parameter of networkx.louvain_communities.
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html
# higher values favor smaller clusters of detections corresponding to one bird
LOUVAIN_RESOLUTION = 2
# The length in meters of the lines for visualizing the rays
VIS_RAY_LENGTH_METERS = 100

# Projection detections to mesh
Here we take the per-image detections and project them onto the faces of the mesh. If requested, these projections can be visualized and/or saved to a file for further processing. Note that you could also could convert them to geospatial coordinates using this function, but it's split into two function calls to demonstrate the functionality.

In [ ]:
project_detections(
    mesh_filename=MESH_FILENAME,
    cameras_filename=CAMERAS_FILENAME,
    image_folder=IMAGE_FOLDER,
    detections_folder=DETECTIONS_FOLDER,
    projections_to_mesh_filename=PROJECTIONS_TO_MESH_FILENAME,
    projections_to_geospatial_savefilename=PROJECTIONS_TO_GEOSPATIAL_SAVEFILENAME,
    default_focal_length=DEFAULT_FOCAL_LENGTH,
    project_to_mesh=PROJECT_TO_MESH,
    segmentor_kwargs=SEGMENTOR_KWARGS,
    vis_mesh=VIS_MESH,
)

# Convert mesh projections to geospatial
In the previous step, the per-image detections were projected to the mesh. Now, they are converted to a 2D, geospatial representation. This can be visualized and/or exported as desired.

In [ ]:
project_detections(
    mesh_filename=MESH_FILENAME,
    cameras_filename=CAMERAS_FILENAME,
    image_folder=IMAGE_FOLDER,
    detections_folder=DETECTIONS_FOLDER,
    projections_to_mesh_filename=PROJECTIONS_TO_MESH_FILENAME,
    projections_to_geospatial_savefilename=PROJECTIONS_TO_GEOSPATIAL_SAVEFILENAME,
    convert_to_geospatial=CONVERT_TO_GEOSPATIAL,
    segmentor_kwargs=SEGMENTOR_KWARGS,
    vis_geodata=VIS_GEODATA,
)

# Show the orthomosaic generated for this site and optionally detections
For assessment, the detections are shown overlaid on the orthomosaic. Both the orthomosaic and exported detections are in geospatial coordinates. 

In [ ]:
# Read in the orthomosaic
ortho = rio.open(ORTHO_FILENAME)

# Create axes for consistency between the two data products
_, ax = plt.subplots()
# Note this can take a while for large rasters, I'm not sure there's a way to downsample using this function
rio.plot.show(ortho, ax=ax)

# If there are projections, visualize those too
if PROJECTIONS_TO_GEOSPATIAL_SAVEFILENAME is not None and PROJECTIONS_TO_GEOSPATIAL_SAVEFILENAME.exists():
    # Read the file
    projected_detections = gpd.read_file(PROJECTIONS_TO_GEOSPATIAL_SAVEFILENAME)
    print(projected_detections)
    # Convert to the same CRS as the ortho
    projected_detections.to_crs(ortho.crs, inplace=True)
    # Plot the detections colored by the detection ID
    # This corresponding to their index in the ordered set of all detections
    projected_detections.plot(INSTANCE_ID_KEY, facecolor="none", ax=ax)

# Alternate approach using triangulation 
This approach does not use the mesh for estimating the 3D locations of the birds. Instead, it casts rays in the direction of the center of each detection. Then, it identifies clusters of pairwise near-intersections between rays as likely locations of birds.

In [ ]:
# Create a set of cameras
camera_set = MetashapeCameraSet(
    camera_file=CAMERAS_FILENAME,
    image_folder=IMAGE_FOLDER,
    default_sensor_params={"cx": 0, "cy": 0, "f": DEFAULT_FOCAL_LENGTH}
)

# Determine the shape of the images, assuming they're all the same
image_shape = imread(list(IMAGE_FOLDER.glob("*.JPG"))[0]).shape[:2]

# Create a detector object that looks up detections from a folder. The predictions should be
# one per image in the DeepForest format.
detector = TabularRectangleSegmentor(
    detection_file_or_folder=DETECTIONS_FOLDER,
    image_folder=IMAGE_FOLDER,
    image_shape=image_shape,
    **SEGMENTOR_KWARGS
)

# Create a pyvista plotter to show both scenes at once
plotter = create_pv_plotter(off_screen=False, force_xvfb=False)

# Load the mesh and plot it
mesh = pv.read(MESH_FILENAME)
plotter.add_mesh(mesh, rgb=True)

# Extract the transform to global coordinates from the cameras filename
transform_to_epsg_4978 = parse_transform_metashape(CAMERAS_FILENAME)

# Identify correspondences between detections and show them
detected_bird_locations = camera_set.triangulate_detections(
    detector=detector,
    transform_to_epsg_4978=transform_to_epsg_4978,
    similarity_threshold_meters=RAY_BASED_SIMILARITY_THRESHOLD_METERS,
    louvain_resolution=LOUVAIN_RESOLUTION,
    plotter=plotter,
    vis_ray_length_meters=VIS_RAY_LENGTH_METERS,
)
# Show the cameras
camera_set.vis(show=True, frustum_scale=0.5, plotter=plotter)

# TODO convert these into geospatial coords
print(f"{len(detected_bird_locations)} birds were detected at the following 3D locations, in (lat, lon, alt) coordinates:\n{detected_bird_locations}")

# Exported the triangulated locations 

In [ ]:
# Transform the array to a list of shapely points. Note that the first two coordinates must be
# switched because of different conventions between pyproj and geopandas
points = [Point(l[1], l[0], l[2]) for l in detected_bird_locations]
# Create a dataframe
points_gdf = gpd.GeoDataFrame(geometry=points, crs=LAT_LON_CRS)
# Export the dataframe
points_gdf.to_file(TRIANGULATED_POINTS_SAVEFILE)

# Show the triangulated locations over the ortho

In [ ]:
# Read in the orthomosaic
ortho = rio.open(ORTHO_FILENAME)

# Create axes for consistency between the two data products
_, ax = plt.subplots()
# Note this can take a while for large rasters, I'm not sure there's a way to downsample using this function
rio.plot.show(ortho, ax=ax)

# If there are projections, visualize those too
if TRIANGULATED_POINTS_SAVEFILE is not None and TRIANGULATED_POINTS_SAVEFILE.exists():
    # Read the file
    triangulated_detections = gpd.read_file(TRIANGULATED_POINTS_SAVEFILE)
    # Convert to the same CRS as the ortho
    triangulated_detections.to_crs(ortho.crs)
    # Show each triangulated point
    triangulated_detections.plot(markersize=5, ax=ax)